In [1]:
import pandas as pd
import numpy as np

In [2]:
map_file = "/home/sur/micropopgen/data/test_data/midas/map.txt"
depth_file = "/home/sur/micropopgen/data/test_data/midas/merged.snps/Streptococcus_sp_60086/snps_depth.txt"
info_file = "/home/sur/micropopgen/data/test_data/midas/merged.snps/Streptococcus_sp_60086/snps_info.txt"
freqs_file = "/home/sur/micropopgen/data/test_data/midas/merged.snps/Streptococcus_sp_60086/snps_freq.txt"
groups = ['Buccal mucosa', 'Tongue dorsum']

In [ ]:
map = pd.read_csv(map_file, sep="\t")
map.head()

In [ ]:
info = pd.read_csv(info_file, sep="\t")
info.head()

In [ ]:
depth = pd.read_csv(depth_file, sep="\t")
depth.head()

In [ ]:
freq = pd.read_csv(freqs_file, sep="\t")
freq.head()

In [ ]:
freq.shape

In [ ]:
depth.shape

In [ ]:
info.shape

In [ ]:
groups

In [ ]:
type(map.Group)

In [79]:
map = pd.read_csv(map_file, sep="\t")
map = map.loc[map.Group.isin(groups),:].copy()

In [80]:
depth_thres = 1

# Read data
info = pd.read_csv(info_file, sep="\t")
depth = pd.read_csv(depth_file, sep="\t")
freq = pd.read_csv(freqs_file, sep="\t")

# Remove non gene sites
ii = ~info.gene_id.isnull()
info = info.loc[ii, :]
depth = depth.loc[ii, :]
freq = freq.loc[ii, :]
# print(info.shape)
# print(depth.shape)
# print(freq.shape)

gene_ids = info.gene_id.unique()

if not all(freq.columns == depth.columns):
    raise ValueError("Columns don't mathc between freq and depth files")
    
# Remove samples from other groups
ci = depth.columns.isin(map.ID.append(pd.Series('site_id')))
depth = depth.loc[:, ci]
freq = freq.loc[:, ci]
# print(info.shape)
# print(depth.shape)
# print(freq.shape)

# subset for tests
info = info.head(20)
depth = depth.head(20)
freq = freq.head(20)



(164077, 16)
(164077, 130)
(164077, 130)


In [119]:
i = 0
# Add samples IDs as map header and match samples
# in map with samples in depth
map.index = map.ID
map = map.loc[depth.columns[1:],:]

if not all(map.index == depth.columns[1:]):
    raise ValueError("Samples in map and depth don't match")

# Create site data frame
site = map.copy()
site['depth'] = depth.loc[ depth.index[i], map.index ]
site['freq'] = freq.loc[ freq.index[i], map.index ]
print(site.head())

# Slice by depth
site = site[site.depth >= depth_thres]
print(site.shape)
print(site.head())
# site

# Determine if it is polymorphic or fixed
site_crosstab = pd.crosstab(site.freq >= 0.5, site.Group)
site_crosstab

                  ID          Group  depth  freq
SRS144368  SRS144368  Tongue dorsum      0   0.0
SRS144512  SRS144512  Tongue dorsum      0   0.0
SRS144609  SRS144609  Tongue dorsum      0   0.0
SRS146738  SRS146738  Tongue dorsum      0   0.0
SRS146767  SRS146767  Tongue dorsum      3   0.0
(42, 4)
                  ID          Group  depth  freq
SRS146767  SRS146767  Tongue dorsum      3   0.0
SRS146894  SRS146894  Tongue dorsum      6   0.0
SRS149671  SRS149671  Tongue dorsum     37   0.0
SRS064774  SRS064774  Tongue dorsum      4   0.0
SRS076951  SRS076951  Tongue dorsum      6   0.0


Group,Buccal mucosa,Tongue dorsum
freq,,
False,1,39
True,0,2


In [170]:
site_crosstab.columns

mutation_type = ''
if (np.matrix(site_crosstab).diagonal() == [0,0]).all() or (np.fliplr(np.matrix(site_crosstab)).diagonal() == [0 0]).all():
    mutation_type = 'fixed'
else:
    mutation_type = 'polymorphic

True

In [137]:
np.matrix(site_crosstab)

matrix([[ 1, 39],
        [ 0,  2]])

In [135]:
np.matrix(site_crosstab).diagonal(axis1=0, axis2=1)

matrix([[1, 2]])

In [136]:
np.matrix(site_crosstab).diagonal(axis1=1, axis2=0)

matrix([[1, 2]])

AttributeError: 'DataFrame' object has no attribute 'diagonal'

In [145]:
np.matrix(site_crosstab)[:,::-1].diagonal()

matrix([[39,  0]])

In [146]:
np.fliplr(np.matrix(site_crosstab)).diagonal()

matrix([[39,  0]])

In [149]:
%timeit np.matrix(site_crosstab)[:,::-1].diagonal()

29.9 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [150]:
%timeit np.fliplr(np.matrix(site_crosstab)).diagonal()

30.7 µs ± 404 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [40]:
a = freq.iloc[0,1:] > 0.5
b = depth.iloc[0,1:] > 1
# print(a)
# print(b)

a = a[0:10]
b = b[0:10]

print(len(a))
print(len(b))

# a = pd.Series([True, True, True, False,
#               False, False])
# b = pd.Series([False, True, False, True,
#               False, True])
print(a)
print(b)




10
10
SRS144368    False
SRS144512    False
SRS144609    False
SRS146738    False
SRS146767    False
SRS146824    False
SRS146894    False
SRS147094    False
SRS147120    False
SRS147226    False
Name: 12, dtype: bool
SRS144368    False
SRS144512    False
SRS144609    False
SRS146738    False
SRS146767     True
SRS146824    False
SRS146894     True
SRS147094    False
SRS147120    False
SRS147226    False
Name: 12, dtype: bool


12

In [46]:
pd.crosstab(a,b, colnames=['b'], rownames=['a'])

b,False,True
a,,
False,8,2


In [84]:
info

,site_id,ref_id,ref_pos,ref_allele,major_allele,minor_allele,count_samples,count_a,count_c,count_g,count_t,locus_type,gene_id,snp_type,site_type,amino_acids
12,419,CM002128,419,A,A,T,42,440,0,0,17,CDS,1316408.3.peg.1,bi,3D,"I,M,I,I"
13,425,CM002128,425,G,G,T,42,1,0,428,9,CDS,1316408.3.peg.1,bi,3D,"I,M,I,I"
14,437,CM002128,437,A,A,T,41,353,0,0,17,CDS,1316408.3.peg.1,bi,4D,"S,S,S,S"
15,443,CM002128,443,A,A,G,38,275,0,37,0,CDS,1316408.3.peg.1,bi,4D,"P,P,P,P"
16,453,CM002128,453,T,T,C,32,0,6,0,259,CDS,1316408.3.peg.1,bi,1D,"I,S,T,N"
17,475,CM002128,475,G,G,A,25,14,0,216,0,CDS,1316408.3.peg.1,bi,2D,"L,V,L,I"
18,476,CM002128,476,A,A,T,24,214,0,0,7,CDS,1316408.3.peg.1,bi,4D,"V,V,V,V"
19,479,CM002128,479,C,C,T,24,0,225,0,7,CDS,1316408.3.peg.1,bi,3D,"N,K,N,K"
20,482,CM002128,482,A,A,T,23,227,0,0,7,CDS,1316408.3.peg.1,bi,4D,"V,V,V,V"
21,491,CM002128,491,T,T,C,21,0,64,0,166,CDS,1316408.3.peg.1,bi,3D,"N,K,N,K"


In [48]:
map.ID.append(pd.Series('site_id'))

2      SRS011086
3      SRS011090
5      SRS011115
8      SRS011140
9      SRS011144
12     SRS011243
13     SRS011247
17     SRS011306
18     SRS011310
25     SRS012279
26     SRS012281
30     SRS012916
31     SRS012918
34     SRS012983
35     SRS012985
39     SRS013164
42     SRS013234
43     SRS013239
46     SRS013502
47     SRS013506
53     SRS013705
54     SRS013711
57     SRS013818
58     SRS013825
60     SRS013879
61     SRS013881
62     SRS013945
65     SRS014124
66     SRS014126
68     SRS014271
         ...    
798    SRS144368
799    SRS144372
802    SRS144494
804    SRS144512
806    SRS144542
808    SRS144560
809    SRS144609
811    SRS144646
813    SRS145503
814    SRS145505
816    SRS146738
819    SRS146767
823    SRS146824
826    SRS146894
827    SRS146979
831    SRS147094
833    SRS147120
834    SRS147122
836    SRS147226
837    SRS147334
839    SRS147436
845    SRS147887
847    SRS147933
849    SRS148150
853    SRS148276
854    SRS148408
859    SRS149671
860    SRS1499

In [44]:
mysamples = test.ID
mysamples.append(pd.Series('test'))
test.ID.append(pd.Series('test'))
test

,ID,Group
2,SRS011086,Tongue dorsum
3,SRS011090,Buccal mucosa
5,SRS011115,Tongue dorsum
8,SRS011140,Tongue dorsum
9,SRS011144,Buccal mucosa


In [22]:
depth.head()

,site_id,SRS144368,SRS144512,SRS144609,SRS146738,SRS146767,SRS146824,SRS146894,SRS147094,SRS147120,...,SRS018791,SRS018969,SRS019022,SRS019045,SRS019894,SRS020220,SRS021496,SRS021954,SRS022530,SRS022621
12,419,0,0,0,0,3,0,6,0,0,...,0,1,6,2,2,4,3,8,0,0
13,425,0,0,0,0,3,0,6,0,0,...,0,0,5,4,2,4,3,11,0,0
14,437,0,0,0,0,3,0,6,0,0,...,0,0,5,2,2,4,4,6,0,0
15,443,0,0,0,0,3,0,3,0,0,...,0,0,3,2,1,3,4,5,0,0
16,453,0,0,0,0,0,0,2,0,0,...,0,0,3,2,1,1,2,6,0,0


In [23]:
freq.head()

,site_id,SRS144368,SRS144512,SRS144609,SRS146738,SRS146767,SRS146824,SRS146894,SRS147094,SRS147120,...,SRS018791,SRS018969,SRS019022,SRS019045,SRS019894,SRS020220,SRS021496,SRS021954,SRS022530,SRS022621
12,419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
13,425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
14,437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
15,443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.0,0.0
16,453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


True